In [38]:
import sqlite3 as sql

import config
import mysql.connector
import numpy as np
import pandas as pd
from mysql.connector import Error


def connect_to_db(config: dict[str, str]):
    return mysql.connector.connect(**config)


def query_db(query: str, print_query: bool = True):
    if print_query:
        print(query)

    records = pd.DataFrame()

    try:
        connection = connect_to_db(config.maria_database_config)
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            cursor.execute(query)

            rows_affected = cursor.rowcount
            if rows_affected > 0:
                print(f"Rows affected: {rows_affected}")

            records = pd.DataFrame(cursor.fetchall())

    except Error as error_:
        print("Error while connecting to MySQL", error_)

    finally:

        if connection.is_connected():
            cursor.close()
            connection.close()

    return records

In [39]:
# check if everything works
query = '''\
SELECT *
FROM artist;
'''
query_db(query).head()

SELECT *
FROM artist;



,artist_id,full_name,first_name,middle_names,last_name,nationality,style,birth,death
0,500,Pierre-Auguste Renoir,Pierre,Auguste,Renoir,French,Impressionist,1841,1919
1,501,Alexandre Cabanel,Alexandre,,Cabanel,French,Classicist,1823,1889
2,502,James Ensor,James,,Ensor,Belgian,Expressionist,1860,1949
3,503,Maximilien Luce,Maximilien,,Luce,French,Pointillist,1858,1941
4,504,August Macke,August,,Macke,German,Expressionist,1887,1914


In [40]:
# Famous Paintings Challenge
# Überblick und einfache Queries, numerische Funktionen:
# 1.) Liste alle Künstler und ihre Geburtsdaten auf.
query = '''\
SELECT full_name, birth
FROM artist;
'''
query_db(query).head()

SELECT full_name, birth
FROM artist;



,full_name,birth
0,Pierre-Auguste Renoir,1841
1,Alexandre Cabanel,1823
2,James Ensor,1860
3,Maximilien Luce,1858
4,August Macke,1887


In [41]:
# Famous Paintings Challenge
# Überblick und einfache Queries, numerische Funktionen:
# 2.) Zeige alle Museen in Deutschland an.
query = '''\
SELECT name, country
FROM museum
WHERE country="Germany";
'''
query_db(query).head()

SELECT name, country
FROM museum
WHERE country="Germany";



,name,country
0,Museum Folkwang,Germany


In [42]:
# Famous Paintings Challenge
# Überblick und einfache Queries, numerische Funktionen:
# 3.) Liste alle Künstler auf, die nach 1800 geboren wurden, und sortiere
# die Liste nach dem Geburtsjahr absteigend.
query = '''\
SELECT full_name, birth
FROM artist
WHERE birth > 1800
ORDER BY birth DESC;
'''
query_db(query).head()

SELECT full_name, birth
FROM artist
WHERE birth > 1800
ORDER BY birth DESC;



,full_name,birth
0,Mark Rothko,1903
1,Salvador Dalí,1901
2,Yves Tanguy,1900
3,Chaïm Soutine,1893
4,Grant Wood,1891


In [43]:
# Famous Paintings Challenge
# Überblick und einfache Queries, numerische Funktionen:
# 4.) Zähle die Anzahl der Gemälde für jeden Kunststil (Style) und gib die
# Ergebnisse in aufsteigender Reihenfolge aus.
query = '''\
SELECT COUNT(work_id), style
FROM work
GROUP BY style
ORDER BY COUNT(work_id) ASC;
'''
query_db(query).head()

SELECT COUNT(work_id), style
FROM work
GROUP BY style
ORDER BY COUNT(work_id) ASC;



,COUNT(work_id),style
0,70,Japanese Art
1,108,Art Nouveau
2,146,Avant-Garde
3,149,Surrealism
4,198,Symbolism


In [44]:
# Famous Paintings Challenge
# Überblick und einfache Queries, numerische Funktionen:
# 5.) Zeige alle Arbeiten (work) an, die im Museum mit der ID 30
# ausgestellt sind.
query = '''\
SELECT name
FROM work
WHERE museum_id = 30;
'''
query_db(query).head()

SELECT name
FROM work
WHERE museum_id = 30;



,name
0,Archers
1,Bride and Groom (The Newlyweds)
2,Veranda in Capri
3,At the Milliner's
4,Subway riders in NYC


In [45]:
# Famous Paintings Challenge
# M.1.) Berechne den durchschnittlichen Todestag (Alter) aller Künstler
query = '''\
SELECT AVG(death - birth) AS average_age_at_death
FROM artist;
'''
query_db(query).head()

SELECT AVG(death - birth) AS average_age_at_death
FROM artist;



,average_age_at_death
0,66.3325


In [46]:
# Famous Paintings Challenge
# Datentypen, Subqueries:
# Zeige alle Museen an, die sich in New York oder im Vereinten
# Königreich befinden.
query = '''\
SELECT name, IF(country = "United Kingdom", "United Kingdom", IF(city = "New York", "New York", "")) AS place
FROM museum
WHERE country = "United Kingdom"
    OR city = "New York";
'''
query_db(query).head()

SELECT name, IF(country = "United Kingdom", "United Kingdom", IF(city = "New York", "New York", "")) AS place
FROM museum
WHERE country = "United Kingdom"
    OR city = "New York";



,name,place
0,The Museum of Modern Art,New York
1,The Metropolitan Museum of Art,New York
2,National Maritime Museum,United Kingdom
3,Solomon R. Guggenheim Museum,New York


In [47]:
# Famous Paintings Challenge
# Datentypen, Subqueries:
# Liste alle Künstler und die Anzahl der von ihnen erstellten Gemälde auf.
# TODO: Frage: Geht das auch ohne JOIN?
query = '''\
SELECT artist.full_name, COUNT(work.artist_id) AS paintings_count
FROM artist
INNER JOIN work ON work.artist_id = artist.artist_id
GROUP BY work.artist_id;
'''
query_db(query).head()

SELECT artist.full_name, COUNT(work.artist_id) AS paintings_count
FROM artist
INNER JOIN work ON work.artist_id = artist.artist_id
GROUP BY work.artist_id;



,full_name,paintings_count
0,Pierre-Auguste Renoir,469
1,Alexandre Cabanel,13
2,James Ensor,3
3,Maximilien Luce,174
4,August Macke,68


In [48]:
# JOINS und weitere Funktionen:
# Liste alle Künstler auf, die Gemälde im Stil "Baroque" erstellt haben.
query = '''\
SELECT artist.full_name
FROM artist
INNER JOIN work ON work.artist_id = artist.artist_id
WHERE work.style = "Baroque"
GROUP BY artist.full_name;
'''
query_db(query).head()

SELECT artist.full_name
FROM artist
INNER JOIN work ON work.artist_id = artist.artist_id
WHERE work.style = "Baroque"
GROUP BY artist.full_name;



,full_name
0,Aelbert Cuyp
1,Albrecht Adam
2,Bartolomé Esteban Murillo
3,Caesar Van Everdingen
4,Caravaggio


In [49]:
# JOINS und weitere Funktionen:
# Zeige alle Museen und ihre Öffnungszeiten an, auch wenn das Museum keine
# definierten Öffnungszeiten hat.
query = '''\
SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum_hours.museum_id = museum.museum_id;
'''
query_db(query).head()

SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum_hours.museum_id = museum.museum_id;



,name,day,open,close
0,The Museum of Modern Art,Sunday,10:30:AM,05:30:PM
1,The Museum of Modern Art,Monday,10:30:AM,05:30:PM
2,The Museum of Modern Art,Tuesday,10:30:AM,05:30:PM
3,The Museum of Modern Art,Wednesday,10:30:AM,05:30:PM
4,The Museum of Modern Art,Thusday,10:30:AM,05:30:PM


In [50]:
# JOINS und weitere Funktionen:
# Liste die Künstler auf, die nach einem bestimmten Jahr (1800) geboren
# wurden und Gemälde erstellt haben.
query = '''\
SELECT full_name, birth
FROM artist
WHERE birth > 1800
    AND EXISTS (
        SELECT name
        FROM work
        WHERE work.artist_id = artist.artist_id);
'''
query_db(query).head()

SELECT full_name, birth
FROM artist
WHERE birth > 1800
    AND EXISTS (
        SELECT name
        FROM work
        WHERE work.artist_id = artist.artist_id);



,full_name,birth
0,Pierre-Auguste Renoir,1841
1,Alexandre Cabanel,1823
2,James Ensor,1860
3,Maximilien Luce,1858
4,August Macke,1887


In [51]:
# JOINS und weitere Funktionen:
# Zeige alle Künstler an, die Gemälde im Museum „The Museum of Modern Art“
# erstellt haben.
query = '''\
SELECT artist.full_name
FROM artist
WHERE EXISTS (
    SELECT work.name
    FROM work
    INNER JOIN museum ON museum.museum_id = work.museum_id
    WHERE museum.name = "The Museum of Modern Art");
'''
query_db(query).head()

SELECT artist.full_name
FROM artist
WHERE EXISTS (
    SELECT work.name
    FROM work
    INNER JOIN museum ON museum.museum_id = work.museum_id
    WHERE museum.name = "The Museum of Modern Art");



,full_name
0,Pierre-Auguste Renoir
1,Alexandre Cabanel
2,James Ensor
3,Maximilien Luce
4,August Macke


In [52]:
# JOINS und weitere Funktionen:
# Liste alle Künstler auf, die im Verzeichnis "National Gallery"
# ausgestellt haben.
query = '''\
SELECT artist.full_name
FROM artist
WHERE EXISTS (
    SELECT work.name
    FROM work
    INNER JOIN museum ON museum.museum_id = work.museum_id
    WHERE museum.name = "National Gallery");
'''
query_db(query).head()

SELECT artist.full_name
FROM artist
WHERE EXISTS (
    SELECT work.name
    FROM work
    INNER JOIN museum ON museum.museum_id = work.museum_id
    WHERE museum.name = "National Gallery");



,full_name
0,Pierre-Auguste Renoir
1,Alexandre Cabanel
2,James Ensor
3,Maximilien Luce
4,August Macke


In [53]:
# Import/Export von Datenbanken, Constraints:
# Lösche alle Informationen über Gemälde im Museum mit der ID 32.
query = '''\
DELETE FROM work
WHERE museum_id = 32;
'''
query_db(query).head()

DELETE FROM work
WHERE museum_id = 43;



""


In [63]:
# Import/Export von Datenbanken, Constraints:
# Erstelle eine Tabelle mit den eindeutigen Künstlernamen.
query = '''\
CREATE TABLE distinct_artist_names
SELECT DISTINCT full_name as distinct_artist_name
FROM artist;

SELECT DISTINCT full_name as distinct_artist_name
INTO distinct_artist_names
FROM artist;
'''
query_db(query).head()

CREATE TABLE distinct_artist_names
SELECT DISTINCT full_name as distinct_artist_name
FROM artist;

SELECT DISTINCT full_name as distinct_artist_name
INTO distinct_artist_names
FROM artist;

Rows affected: 421


""


In [66]:
# Import/Export von Datenbanken, Constraints:
# Füge der Tabelle "artist" eine neue Spalte "artist_key" hinzu und
# weise sie als Primärschlüssel zu. Die "artist_key" soll automatisch
# inkrementiert werden.
query = '''\
ALTER TABLE artist
ADD artist_key INT PRIMARY KEY AUTO_INCREMENT;
'''
query_db(query).head()

ALTER TABLE artist
ADD artist_key INT PRIMARY KEY AUTO_INCREMENT;



""


In [68]:
# Import/Export von Datenbanken, Constraints:
# Erstelle eine View mit dem Namen "view_artists" basierend auf der
# Tabelle "artist", die nur die Spalten "full_name" und "style" enthält.
query = '''\
CREATE VIEW view_artists AS
SELECT full_name, style
FROM artist;
'''
query_db(query).head()

CREATE VIEW view_artists AS
SELECT full_name, style
FROM artist;



""


In [86]:
# Import/Export von Datenbanken, Constraints:
# Ändere die Öffnungszeiten eines Museums für einen bestimmten Tag
# („Musée du Louvre“, Montag 09 bis 19Uhr).

query = '''\
SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum.museum_id = museum_hours.museum_id
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";
'''
print(query_db(query).head(), "\n")

query = '''\
UPDATE museum_hours
INNER JOIN museum ON museum.museum_id = museum_hours.museum_id
SET museum_hours.open = "09:00:AM", museum_hours.close = "07:00:PM"
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";
'''
print(query_db(query).head(), "\n")

query = '''\
SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum.museum_id = museum_hours.museum_id
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";
'''
print(query_db(query).head(), "\n")

SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum.museum_id = museum_hours.museum_id
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";

              name     day      open     close
0  Musée du Louvre  Monday  09:00:AM  07:00:PM 

UPDATE museum_hours
INNER JOIN museum ON museum.museum_id = museum_hours.museum_id
SET museum_hours.open = "09:00:AM", museum_hours.close = "07:00:PM"
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";

Empty DataFrame
Columns: []
Index: [] 

SELECT museum.name, museum_hours.day, museum_hours.open, museum_hours.close
FROM museum
INNER JOIN museum_hours ON museum.museum_id = museum_hours.museum_id
WHERE museum.name = "Musée du Louvre"
    AND museum_hours.day = "Monday";

              name     day      open     close
0  Musée du Louvre  Monday  09:00:AM  07:00:PM 



In [99]:
# Import/Export von Datenbanken, Constraints:
# - Füge einen Index mit Namen idx_artist_id zur Spalte artist_id
# in der Tabelle artist hinzu.
# - Erstelle einen Fremdschlüssel, der auf die Spalte artist_id
# in der Tabelle artist verweist.
query = '''\
CREATE INDEX idx_artist_id ON artist(artist_id);
'''
query_db(query).head()


query = '''\
ALTER TABLE distinct_artist_names
ADD artist_id INT;
'''
query_db(query).head()

query = '''\
ALTER TABLE distinct_artist_names
ADD FOREIGN KEY (artist_id) REFERENCES artist(artist_id);
'''
query_db(query).head()

query = '''\
UPDATE distinct_artist_names
SET artist_id = (
    SELECT artist_id
    FROM artist
    WHERE artist.full_name = distinct_artist_names.distinct_artist_name
)
'''
query_db(query).head()

CREATE INDEX idx_artist_id ON artist(artist_id);

Error while connecting to MySQL 1061 (42000): Duplicate key name 'idx_artist_id'
ALTER TABLE distinct_artist_names
ADD artist_id INT;

Error while connecting to MySQL 1060 (42S21): Duplicate column name 'artist_id'
ALTER TABLE distinct_artist_names
ADD FOREIGN KEY (artist_id) REFERENCES artist(artist_id);

Rows affected: 421
UPDATE distinct_artist_names
SET artist_id = (
    SELECT artist_id
    FROM artist
    WHERE artist.full_name = distinct_artist_names.distinct_artist_name
)

Rows affected: 421


""
